In [71]:
import statsmodels
import scipy
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import *
from statsmodels.sandbox.stats.multicomp import multipletests 

In [33]:
def waf(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(str(round(answer, 3)))

In [2]:
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

1. для групп normal (control) и early neoplasia (treatment)
* для групп early neoplasia (control) и cancer (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [4]:
data.Diagnosis.value_counts()

early neoplasia    25
normal             24
cancer             23
Name: Diagnosis, dtype: int64

In [316]:
norm = data.iloc[:, 2:][(data.Diagnosis == 'normal')]
early_neoplasia = data.iloc[:, 2:][(data.Diagnosis == 'early neoplasia')]
cancer = data.iloc[:, 2:][(data.Diagnosis) == 'cancer']

In [317]:
norm_vs_early_m = {}
norm_vs_early_p = {}
for i in data.columns[2:]:
    z = scipy.stats.ttest_ind(norm[i], early_neoplasia[i], equal_var = False)
    norm_vs_early_p[i] = z[1]
    if z[1] <= 0.05:
        norm_vs_early_m[i] = z[1]

In [318]:
early_vs_cancer_m = {}
early_vs_cancer_p = {}
for i in data.columns[2:]:
    z = scipy.stats.ttest_ind(early_neoplasia[i], cancer[i], equal_var = False)
    early_vs_cancer_p[i] = z[1]
    if z[1] <= 0.05:
        early_vs_cancer_m[i] = z[1]

In [363]:
waf(len(norm_vs_early_m), 'genes_answer_1.txt')
waf(len(early_vs_cancer_m), 'genes_answer_2.txt')

### Часть 2: поправка методом Холма
Для этой части задания вам понадобится модуль multitest из statsmodels.
import statsmodels.stats.multitest as smm
В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.51.5.

Обратите внимание, что

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!

In [337]:
p_norm_vs_early = pd.DataFrame({'norm_vs_early': list(norm_vs_early_p.values())})
p_early_vs_cancer = pd.DataFrame({'early_vs_cancer':list(early_vs_cancer_p.values())})

In [338]:
reject, p_corrected, _, _ = multipletests(p_norm_vs_early['norm_vs_early'], 
                                            alpha = 0.05/2, method = 'holm')
p_norm_vs_early['p_corrected'] = p_corrected
p_norm_vs_early['reject'] = reject

reject, p_corrected, _, _ = multipletests(p_early_vs_cancer['early_vs_cancer'], 
                                            alpha = 0.05/2, method = 'holm')
p_early_vs_cancer['p_corrected'] = p_corrected
p_early_vs_cancer['reject'] = reject

In [347]:
def fold_change_counter(list_with_indices, array1, array2):
    """
    Counts folds along statistically important entries
    Takes array indices column-wise
    """
    
    what_really_matters = {}
    for i in list_with_indices:
        a = np.mean(array1.iloc[:, i])
        b = np.mean(array2.iloc[:, i])
        z = abs(a/b if a > b else b / a)
        if z > 1.5:
            what_really_matters[i] = z
    return what_really_matters

Практически значимые различия в группах:

In [362]:
differences_norm_vs_neoplastia = fold_change_counter(np.where(p_norm_vs_early.reject == True)[0], 
                   norm, early_neoplasia)
differences_neoplastia_cancer = fold_change_counter(np.where(p_early_vs_cancer.reject == True)[0], 
                   early_neoplasia, cancer)
print('Quantity of differences in norm vs neoplastia: %d' % len(differences_norm_vs_neoplastia))
print('Quantity of differences in neoplastia vs cancer: %d' % len(differences_neoplastia_cancer))

Quantity of differences in norm vs neoplastia: 2
Quantity of differences in neoplastia vs cancer: 77


In [364]:
waf(len(differences_norm_vs_neoplastia), 'genes_answer_3.txt')
waf(len(differences_neoplastia_cancer), 'genes_answer_4.txt')

### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$
​	 , когда они есть, и будут чаще отклонять $H_0$	 , когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.

In [367]:
p_norm_vs_early = pd.DataFrame({'norm_vs_early': list(norm_vs_early_p.values())})
p_early_vs_cancer = pd.DataFrame({'early_vs_cancer':list(early_vs_cancer_p.values())})

In [368]:
reject, p_corrected, _, _ = multipletests(p_norm_vs_early['norm_vs_early'], 
                                            alpha = 0.05/2, method = 'fdr_bh')
p_norm_vs_early['p_corrected'] = p_corrected
p_norm_vs_early['reject'] = reject

reject, p_corrected, _, _ = multipletests(p_early_vs_cancer['early_vs_cancer'], 
                                            alpha = 0.05/2, method = 'fdr_bh')
p_early_vs_cancer['p_corrected'] = p_corrected
p_early_vs_cancer['reject'] = reject

In [369]:
differences_norm_vs_neoplastia = fold_change_counter(np.where(p_norm_vs_early.reject == True)[0], 
                   norm, early_neoplasia)
differences_neoplastia_cancer = fold_change_counter(np.where(p_early_vs_cancer.reject == True)[0], 
                   early_neoplasia, cancer)
print('Quantity of differences in norm vs neoplastia: %d' % len(differences_norm_vs_neoplastia))
print('Quantity of differences in neoplastia vs cancer: %d' % len(differences_neoplastia_cancer))

Quantity of differences in norm vs neoplastia: 4
Quantity of differences in neoplastia vs cancer: 524


In [370]:
waf(len(differences_norm_vs_neoplastia), 'genes_answer_5.txt')
waf(len(differences_neoplastia_cancer), 'genes_answer_6.txt')

In [371]:
p_early_vs_cancer

,early_vs_cancer,p_corrected,reject
0,0.413735,0.675195,False
1,0.653429,0.836406,False
2,0.079556,0.288873,False
3,0.287581,0.563007,False
4,0.463292,0.712214,False
5,0.007681,0.073759,False
6,0.481306,0.724319,False
7,0.578830,0.789177,False
8,0.000740,0.017556,True
9,0.712687,0.867274,False
